In [43]:
import pandas as pd
import requests

In [44]:
inflation_rate = .067

lastest_year = '2017/acs'

oldest_year = '2012'

In [45]:
years = ['2012', '2017']
top_geogs = ['us:1', 'state:34']

In [46]:
def create_top_dfs():
    my_data = {}
    for n in years:
        for d in top_geogs:
            url = 'https://api.census.gov/data/'+ n +'/acs/acs5?get=B00001_001E,NAME&for=' + d +  '&key=37e80b2543b3d018cfbeb06c5dca27cf33e789e7'
            r = requests.get(url)
            x = r.json()
            df = pd.DataFrame(x[1:],columns=x[0])
            raw_result = pd.to_numeric(df['B00001_001E'][0])
            my_data[n + d] = raw_result
    return my_data

In [47]:
top_df_dict = create_top_dfs()

In [48]:
us_previous = top_df_dict['2012us:1']
us_latest = top_df_dict['2017us:1']
nj_previous = top_df_dict['2012state:34']
nj_latest = top_df_dict['2017state:34']

In [49]:
def create_local_dfs(year, geography):
    df_list = []
    url = 'https://api.census.gov/data/'+ year +'/acs5?get=NAME,B00001_001E&for=' + geography +  '&in=state:34&key=37e80b2543b3d018cfbeb06c5dca27cf33e789e7'
    print url
    r = requests.get(url)
    x = r.json()
    df = pd.DataFrame(x[1:], columns=x[0])
    df['B00001_001E'] = df.B00001_001E.replace(to_replace="\+", value='', regex=True)
    df['B00001_001E'] = pd.to_numeric(df.B00001_001E, errors='coerce')
    return df

In [50]:
def clean_local_dfs(year, label):
#     for n in years:
        muni = create_local_dfs(year, 'county%20subdivision:*')
        cty = create_local_dfs(year, 'county:*')
        median_income = muni.append(cty).reset_index().drop(labels='index', axis=1)
        median_income['county subdivision'] = median_income['county subdivision'].fillna('')
        median_income['GEO.id2'] = median_income.state + median_income.county + median_income['county subdivision']
        median_income = median_income[[u'B00001_001E', u'NAME',
                  u'GEO.id2']]
        median_income['GEOLevel1'] = median_income['NAME'].str.split(', ', expand=True)[0]
        median_income['GEOLevel2'] = median_income['NAME'].str.split(', ', expand=True)[1]
        median_income.columns = ['Population_'+ label, 'GEOdisplaylabel', 'GEO.id2','GEOLevel1', 'GEOLevel2']
        median_income['GEOdisplaylabel'] = median_income['GEOdisplaylabel'].replace(to_replace=', New Jersey', value='', regex=True)
        return median_income

In [51]:
pop_latest = clean_local_dfs(lastest_year, 'latest')

https://api.census.gov/data/2017/acs/acs5?get=NAME,B00001_001E&for=county%20subdivision:*&in=state:34&key=37e80b2543b3d018cfbeb06c5dca27cf33e789e7
https://api.census.gov/data/2017/acs/acs5?get=NAME,B00001_001E&for=county:*&in=state:34&key=37e80b2543b3d018cfbeb06c5dca27cf33e789e7


In [52]:
pop_prev = clean_local_dfs(oldest_year, 'prev')

https://api.census.gov/data/2012/acs5?get=NAME,B00001_001E&for=county%20subdivision:*&in=state:34&key=37e80b2543b3d018cfbeb06c5dca27cf33e789e7
https://api.census.gov/data/2012/acs5?get=NAME,B00001_001E&for=county:*&in=state:34&key=37e80b2543b3d018cfbeb06c5dca27cf33e789e7


In [53]:
#Merge Household Income
pop_merge = pd.merge(pop_latest, pop_prev, left_on='GEO.id2', right_on='GEO.id2', how='outer')

In [54]:
pop_merge.head()

,Population_latest,GEOdisplaylabel_x,GEO.id2,GEOLevel1_x,GEOLevel2_x,Population_prev,GEOdisplaylabel_y,GEOLevel1_y,GEOLevel2_y
0,1830.0,"West Windsor township, Mercer County",3402180240,West Windsor township,Mercer County,2011.0,"West Windsor township, Mercer County",West Windsor township,Mercer County
1,1442.0,"East Windsor township, Mercer County",3402119780,East Windsor township,Mercer County,1588.0,"East Windsor township, Mercer County",East Windsor township,Mercer County
2,526.0,"Hopewell borough, Mercer County",3402133150,Hopewell borough,Mercer County,554.0,"Hopewell borough, Mercer County",Hopewell borough,Mercer County
3,253.0,"Hightstown borough, Mercer County",3402131620,Hightstown borough,Mercer County,332.0,"Hightstown borough, Mercer County",Hightstown borough,Mercer County
4,1255.0,"Hopewell township, Mercer County",3402133180,Hopewell township,Mercer County,1190.0,"Hopewell township, Mercer County",Hopewell township,Mercer County


In [55]:
pop_merge = pop_merge.drop(['GEOdisplaylabel_y', 'GEOLevel1_x', 'GEOLevel2_x'], axis=1)
pop_merge.rename(columns={'GEOdisplaylabel_x': 'GEOdisplaylabel', 'GEOLevel1_y': 'GEOLevel1','GEOLevel2_y':'GEOLevel2', 'GEO.id_x': 'GEO.id', 'GEOdisplaylabel_long_x': 'GEOdisplaylabel_long'}, inplace=True)

In [56]:
#Take out County subdivisions not defined
pop_merge = pop_merge[pop_merge['GEOdisplaylabel'].str.contains("County subdivisions not defined") == False]
pop_merge = pop_merge.sort_values(by=['GEO.id2'])

In [57]:
pop_merge.head()

,Population_latest,GEOdisplaylabel,GEO.id2,Population_prev,GEOLevel1,GEOLevel2
577,21206.0,Atlantic County,34001,20700.0,Atlantic County,New Jersey
98,626.0,"Absecon city, Atlantic County",3400100100,634.0,Absecon city,Atlantic County
121,3090.0,"Atlantic City city, Atlantic County",3400102080,2730.0,Atlantic City city,Atlantic County
109,887.0,"Brigantine city, Atlantic County",3400107810,746.0,Brigantine city,Atlantic County
112,348.0,"Buena borough, Atlantic County",3400108680,324.0,Buena borough,Atlantic County


In [58]:
#Add NJ and Nationwide
pop_merge['nj_pop_latest'] = nj_latest
pop_merge['nj_pop_prev'] = nj_previous
pop_merge['usa_pop_latest'] = us_latest
pop_merge['usa_pop_prev'] = us_previous
pop_merge['pop_per_change'] = ((pop_merge['Population_latest'] - pop_merge['Population_prev']) / pop_merge['Population_prev']) * 100
pop_merge['nj_pop_change'] = ((pop_merge['nj_pop_latest'] - pop_merge['nj_pop_prev']) / pop_merge['nj_pop_prev']) * 100
pop_merge['us_pop_change'] = ((pop_merge['usa_pop_latest'] - pop_merge['usa_pop_prev']) / pop_merge['usa_pop_prev']) * 100


In [60]:
pop_merge = pop_merge[[ u'GEOdisplaylabel', u'GEO.id2',
        u'GEOLevel1', u'GEOLevel2',u'Population_latest', 
         u'Population_prev',
       u'nj_pop_latest', u'nj_pop_prev',
       u'usa_pop_latest', u'usa_pop_prev',
       u'pop_per_change', u'nj_pop_change',
       u'us_pop_change']]

In [61]:
pop_merge.head()

,GEOdisplaylabel,GEO.id2,GEOLevel1,GEOLevel2,Population_latest,Population_prev,nj_pop_latest,nj_pop_prev,usa_pop_latest,usa_pop_prev,pop_per_change,nj_pop_change,us_pop_change
577,Atlantic County,34001,Atlantic County,New Jersey,21206.0,20700.0,681807,655989,26328166,24458387,2.444444,3.935737,7.644736
98,"Absecon city, Atlantic County",3400100100,Absecon city,Atlantic County,626.0,634.0,681807,655989,26328166,24458387,-1.261830,3.935737,7.644736
121,"Atlantic City city, Atlantic County",3400102080,Atlantic City city,Atlantic County,3090.0,2730.0,681807,655989,26328166,24458387,13.186813,3.935737,7.644736
109,"Brigantine city, Atlantic County",3400107810,Brigantine city,Atlantic County,887.0,746.0,681807,655989,26328166,24458387,18.900804,3.935737,7.644736
112,"Buena borough, Atlantic County",3400108680,Buena borough,Atlantic County,348.0,324.0,681807,655989,26328166,24458387,7.407407,3.935737,7.644736


In [62]:
# pop_merge.to_csv('pop_data_towns_clean.csv', header=True, index=False)